## Описание задачи 📕

1) Найти на портале ГИС ЖКХ (https://dom.gosuslugi.ru/#!/main) данные о количестве платежей ЖКХ по субъектам РФ
2) Найти на портале данные о тарифах ЖКХ
3) Сформулировать правила расчета и рассчитать приблизительный средний тариф ЖКХ по субъектам РФ за 2023 год по данным ГИС ЖКХ и/или другим источникам 
4) Определить объем рынка ЖКХ в рублях по субъектам РФ и в целом в РФ

## Результат ✅

1) Оценка объема рынка ЖКХ по субъектам РФ и в целом в РФ
2) Файл в формате ipynb с кодом расчета объема рынка ЖКХ в рублях и коротким описанием методологии расчета

## Алгоритм решения задачи 👨‍💻

### Расчет средних выплат по ЖКХ для субъекта РФ

1) Определить те тарифы, которые будут использоваться в данном ислледовании. **Обоснование**: средний житель страны платит лишь по 6-7 тарифам
2) Выявить средний размер домохозяйства для выбранного субъекта: кол-во человек в домохозяйстве и размер жилой площади. **Обоснование**: эти показатели мы будем перемножать с тарифом на услугу и на норму потребления, чтобы получить стоимость общих выплат по ЖКХ 
3) Выявить процент населения, живущего в многоквартирных домах. **Обоснование**: не все платежи приспособлены для индивидуальных домах, где в среднем располагаются от 15-ти до 20-ти процентов об общего населения
4) Пусть $a_1, a_2 ... a_n$ - различные тарифы по ЖКХ, $b_1, b_2 ... b_n$ - потребление услуг ЖКХ на квадратный метр жилья либо на человека, $c$ - показатель среднего размера семьи/жил.площади, $d$ - показатель доли населения, живущих в многоквартирных домах, $e_1, e_2 ... e_n$ - тарифы ЖКХ, предназначенные только жителям квартирных домов. Тогда, для расчета средних выплат по ЖКХ для одного домохозяйства будет использоваться следующая формула:
$a_1*b_1*c + a_2*b_2*c ... + a_n*b_n*c + e_1*d+e_2*d ... + e_n*d$




### Расчет средних выплат по ЖКХ для РФ
1) Выявить население в каждом субъекте РФ. **Обоснование**: средние выплаты по ЖКХ для одного домохозяйства в РФ будут рассчитаны как средневзвешенные средние выплаты по ЖКХ для одного домохозяйства по субъектам РФ, где в качестве веса будет использоваться население субъекта
2) Пусть $a_1, a_2 ... a_n$ - средние выплаты по ЖКХ для одного домохозяйства в субъектах РФ, $b_1, b_2 ... b_n$ - население субъекта РФ. Тогда, для расчета средних выплат по ЖКХ для одного домохозяйства в РФ будет использоваться следующая формула:
$\frac{a_1*b_1 + a_2*b_2 ... + a_n*b_n}{b_1+b_2 ...+ b_n}$

# ==========================================================

In [313]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

## Загружаем все необходимые данные

#### 1) Тарифы на оплату коммунальных ресурсов
URL: https://dom.gosuslugi.ru/filestore/publicDownloadServlet?context=tariff-exports&uid=14a51197-f631-4337-b7ae-f2c968e96fb1

In [318]:
data_path = 'Data/Тарифы на оплату коммунальных ресурсов от 29.05.2023/'
rate_files = [i for i in os.listdir(data_path) if '.xlsx' in i and '$' not in i] 
rates_df = pd.DataFrame()
for file in tqdm(rate_files):
    df = pd.read_excel(data_path + file)
    df['Регион'] = file.split('.')[0]
    rates_df = pd.concat([rates_df, df])


# Оставляем только нужные колонки
rates_df = rates_df[['Регион', 'Вид тарифа', 'Величина тарифа (ставки тарифа)']].reset_index(drop=True)

100%|███████████████████████████████████████████| 81/81 [01:17<00:00,  1.05it/s]


Преобразование числовых данных:

In [423]:
rate = rates_df['Величина тарифа (ставки тарифа)'].str.split(' ').str[0].astype('float')
metric = rates_df['Величина тарифа (ставки тарифа)'].str.split('/').str[1]
rates_df['Величина тарифа (ставки тарифа)'] = rate
rates_df['Метрика'] = metric

In [424]:
rates_df.head()

,Регион,Вид тарифа,Величина тарифа (ставки тарифа),Метрика
0,Астраханская область,Тариф на техническую воду,36.62,Кубический метр
1,Астраханская область,Тариф на тепловую энергию (мощность),1910.79,Гигакалория
2,Астраханская область,Тариф на водоотведение,29.80,Кубический метр
3,Астраханская область,Тариф на техническую воду,5.93,Кубический метр
4,Астраханская область,Тариф на техническую воду,31.83,Кубический метр


Очистим выбросы данных:

In [462]:
rates_df_clear = pd.DataFrame()
for r in tqdm(rates_df['Регион'].unique()):
    for m in rates_df[rates_df['Регион']==r]['Метрика'].unique():
        for t in rates_df[(rates_df['Метрика']==m) & (rates_df['Регион']==r)]['Вид тарифа'].unique():
            # Обязательно расчет в рамках одной метрики, региона и тарифа
            df = rates_df[(rates_df['Метрика']==m) & (rates_df['Регион']==r) & (rates_df['Вид тарифа']==t)].copy()
            # Находит границы первого и третьего квартиля и считаем разницу между ними
            Q1 = np.percentile(df['Величина тарифа (ставки тарифа)'], 25, method='midpoint')
            Q3 = np.percentile(df['Величина тарифа (ставки тарифа)'], 75, method='midpoint')
            IQR = Q3 - Q1
            # От третьего и первого квартиля движемся вверх и вниз на величину, равную 1.5 IQR, считаем допустимые границы
            upper = Q3 + 1.5*IQR
            lower = Q1 - 1.5*IQR
            # Удаляем выбивающиеся строки
            df = df[df['Величина тарифа (ставки тарифа)'] < upper]
            df = df[df['Величина тарифа (ставки тарифа)'] > lower]
            # Собираем новую таблицу
            rates_df_clear = pd.concat([rates_df_clear, df])

100%|███████████████████████████████████████████| 81/81 [00:57<00:00,  1.40it/s]


In [466]:
rates_df = rates_df_clear.copy()

#### 2) Нормативы потребления коммунальных услуг
URL: https://dom.gosuslugi.ru/rate-consumption/api/rest/services/pconsumptions/downloadzip

In [251]:
data_path = 'Data/Нормативы потребления коммунальных услуг (25.05.2023)/'
cons_files = [i for i in os.listdir(data_path) if '.xlsx' in i and '$' not in i] 
cons_df = pd.DataFrame()
for file in tqdm(cons_files):
    df = pd.read_excel(data_path + file)
    df['Регион'] = file.split(' - ')[0]
    cons_df = pd.concat([cons_df, df])

# Оставляем только нужные колонки, не забыв поставить нужные фильтры
cons_df = cons_df[cons_df['Состояние норматива']=='Действующий'][['Регион', 'Коммунальная услуга', 'Величина норматива']]

100%|███████████████████████████████████████████| 85/85 [00:20<00:00,  4.12it/s]


Преобразование числовых данных:

In [253]:
norm = cons_df['Величина норматива'].str.split(' ').str[0].astype('float')
metric = cons_df['Величина норматива']\
        .apply(lambda x: 'м2' if 'м2' in x.lower() else 'чел' if 'чел' in x.lower() else '')

cons_df['Величина норматива'] = norm
cons_df['Расчет на'] = metric

# Оставляем лишь те строки, где норматив расчитан либо на человека, либо на площадь жилья
cons_df = cons_df[cons_df['Расчет на']!='']

In [254]:
cons_df.head()

,Регион,Коммунальная услуга,Величина норматива,Расчет на
0,Волгоградская область,Отопление,0.010,м2
1,Волгоградская область,Отопление,0.014,м2
2,Волгоградская область,Отопление,0.011,м2
3,Волгоградская область,Отведение сточных вод,1.830,чел
4,Волгоградская область,Отопление,0.016,м2


#### 3) Средний размер домохозяйства по субъектам РФ
URL: https://russia.duck.consulting/maps/264/2010.csv

In [128]:
hh_df = pd.read_csv('Data/Средний_размер_домохозяйства.csv', 
                    header=None, 
                    names=['Год', 'Субъект', 'Значение'])
hh_df = hh_df[hh_df['Год']==2010]

In [137]:
hh_df.head()

,Год,Субъект,Значение
0,2002,Адыгея,2.95
1,2002,Алтайский край,2.69
2,2002,Амурская область,2.74
3,2002,Архангельская область,2.62
4,2002,Астраханская область,2.93


#### 4) Средний тариф на капитальный ремонт по субъектам РФ
Этого тарифа не было в справочнике тарифов на сайте dom.gosuslugi.ru, поэтому выгружаем его из отдельного источника

In [303]:
# Данные были не доступны для скачивания, поэтому парсим таблицу напрямую из html страницы
url = 'https://gogov.ru/articles/utility-rates/overhaul'
headers = {'User-agent': 'Mozilla/5.0'}
rep_df = pd.read_html(requests.get(url,
                               headers=headers).text,
                               attrs={"id":"m-table"})[0]
rep_df = rep_df[['Регион', 'Тариф на капремонт, руб.']]

Преобразование числовых данных:

In [305]:
# Значение для Крыма парсится с ошибкой, установим его вручную
rep_df.loc[rep_df['Регион'] == 'Крым', 'Тариф на капремонт, руб.'] = '7.21'

# Заменим запятые на точки и удалим пробелы
rep_df['Тариф на капремонт, руб.'] = rep_df['Тариф на капремонт, руб.'].str.replace(',', '.').replace(' ', '')

# В случае если два значения через тире, то берет среднее
rate = rep_df['Тариф на капремонт, руб.'].apply(lambda x: float(x) if '-' not in x 
                                         else (float(x.split('-')[0]) + float(x.split(' - ')[1]))/2)
rep_df['Тариф на капремонт, руб.'] = rate

In [306]:
rep_df.head()

,Регион,"Тариф на капремонт, руб."
0,Адыгея,8.70
1,Алтай,10.28
2,Алтайский край,9.93
3,Амурская обл.,8.60
4,Архангельская обл.,11.16


#### 5) Численность населения по субъектам РФ
URL: https://russia.duck.consulting/maps/31/2019.csv

In [255]:
pop_df = pd.read_csv('Data/Численность_населения.csv', 
                    header=None, 
                    names=['Год', 'Субъект', 'Население'])
pop_df = pop_df[pop_df['Год']==2019]

In [256]:
pop_df.head()

,Год,Субъект,Население
2455,2019,Адыгея,454744.0
2456,2019,Алтайский край,2332813.0
2457,2019,Амурская область,793194.0
2458,2019,Архангельская область,1100290.0
2459,2019,Астраханская область,1014065.0


#### 6) Установка констант
Для некоторых показателей не были найдены значения по регионам, поэтому для них мы возьмем некую среднюю оценку, основная на новостной аналитике.

**Средний размер жилой площади**
* Средний размер квартиры - 52.3 м2
* Средний размер дома - 131 м2
* Доля людей, живущих в квартире - 78%, а в доме - 22%
* С учетом долей средний размер жилой площади - 69 м2

**Тарифы, участвующие в анализе** (самые популярные тарифы)
* Отопление
* Электричество
* Водоснабжение
* Водоотведение
* Вывоз мусора
* Взносы на капитальный ремонт (только для квартир)
* Плата за текущий ремонт и содержание (только для квартир)


## Дальнейшие шаги

Дедлайн скоро, поэтому опишу словами, тем более, что по описанию ясно, что расчеты незамысловатые:

* Все значения привести к единому стандарту: регионы и тарифы. Здесь долгое изучение данных в лоб
* Высчитывая средний тариф для региона, я бы взял медиану этого тарифа в рамках всех территорий региона, и таким образом получил бы одно значение на один тариф
* В расчете потребления так же взял медиану этого тарифа в рамках всех территорий региона
* Сджойнить все данные в единую плоскую таблицу
* На основе уже этой таблицы по формулам из описания считать результат
